<span type="title">Java Stand Taglib Library</span> | <span type="update">2018-09-24</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 JSTL。JSTL 是 Java EE 的一个标准标签库，用于提供对于 EL 的逻辑支持。在上一章我们介绍了 Stand Action 以及 EL，这两个标准可以用来创建 Beans 和使用 Beans，以及使用映射，进行一些简单的算术和逻辑运算。但是，这对于完整的提供View层还不够，我们还需要像是条件判断、多项选择、错误处理等等的更像是一门语言的标准。这就是 JSTL。在 JSTL 中使用 EL 表达式能够完成更加丰富的操作。</p></span>

# c:out

c:out 标签用来输出打印值，并且提供一些额外的操作。

```jsp
<h1>Welcome, ${pageScope.person.name}</h1>

<p>${pageScope.person.name}, You are <c:out value="${pageScope.person.age}" default="here" escapeXml="true" /> now.</p>
```

不难看出，C:OUT更加复杂，相比较直接使用EL而言。但是，这样有额外的好处，比如，我们可以使用 default 参数填充默认值，使用 excapeXml 参数来将XML解析为HTML，避免双引号、单引号、尖括号和and号的转义问题（默认开启）。

更为有用的是，c:out 避免了用户输入指令的攻击，防止程序产生预期以外的错误。因此，对于任何可能来自用户输入的地方，都需要强制使用 c:out，而不能直接使用 EL 打印值。c:out 的必选参数是 value，其可以接受运行时参数传入（可以计算得到一个值）。

# c:forEach

这个标签是c库唯一用来遍历容器元素的标签，其可以接受一个iterable的对象，放置在 items 参数中。当标签被执行时，其会在标签作用域创建一个 var 定义好的本地变量，使用指定的 step、start和 end属性来对于 items 的内容进行遍历，每次遍历的值都放在 var 中。注意，此标签还接受一个 varStatus 的指定，这个标签作用域内的本地变量可以在每次遍历时提供当前的指针位置、指针指向的对象。

```jsp
<c:forEach var="word" items="${pageScope.words}" step="1" begin="0" end="3" varStatus="status">
    <p>${status.count} - ${word}</p>
</c:forEach>
```
forEach 可以嵌套，其中被嵌套的标签在外部标签作用域内，可以访问作用域范围的本地变量 group。对下面的例子而言，school是一个列表的列表，group是school的列表，在子循环中，我们用group作为需要被遍历的对象，创建更深的本地变量 item 来取出 group 中的 Person 对象，使用 EL 表达式进行值的获取。

```jsp
<table><c:forEach varStatus="status" var="group" items="${pageScope.school}">
    <tr>
        <c:forEach varStatus="p_status" var="item" items="${group}">
            <td>${item.name}</td>
            <td>${item.age}</td>
            <td>${item.address}</td>
        </c:forEach>
    </tr>
</c:forEach></table>
```

需要注意，所有的标签作用域内的变量都可以使用 ${xxx} 来获取。这是因为，如果不加Scope前缀，默认会使用当前最低等级的作用域，对于 page 而言，就是 pageScope，对于标签而言，则是标签的作用域。出了标签，这个变量将变成 page 作用域下的 Attribute 属性，比如通过 setAttribute 来设置才能访问。

# c:if

这是一个残缺的 IF 条件逻辑，因为没有 else 部分。if 标签接受 test 作为判断逻辑，这个 test 必须可以计算得到一个 bool 值。此外，还可以有 var 属性，这个属性的值将被用作设置为  scope 属性指定的作用域的属性。如果不指定 scope 属性的话，默认是 page 作用域。

**EL表达式和标签属性的作用域推断逻辑不同：** 当标签的属性不设置，则默认为 page，固定死的。当EL表达式不设置作用域，那么从下往上开始查找，找到为止。

```jsp
<c:if test="${param.word eq 'hello'}" var="keys">
    You think the word is hello, you are right!<br/>
</c:if>

<c:if test="${keys eq true}" var="result" scope="page">
    I got key value from pageScope, it is hi.<br/>
</c:if>
${result}
```

# c:choose

为了进行多项选择，使用 choose 和 when。

```jsp
<c:choose>
    <c:when test="${param.word eq 'hi'}">
        You input hi, it is cool <br/>
    </c:when>
    <c:when test="${param.word eq 'hello'}">
        You input hello, it is not bad.
    </c:when>
    <c:otherwise>
        You input ${param.word}. It is just so so.
    </c:otherwise>
</c:choose>
```

注意，这里的 choose 和 switch 不同，如果满足其中一项，则直接全部返回，后面不再执行。你可以理解 choose 和 when 是高级版本的 if - else。when 需要接受一个 test 的布尔值输入，如果符合则进行其体中的语句，否则跳过。otherwise 表示如果都不符合，则执行，类似于最后的 else。


